## contextlib.contextmanager
### デコレータに関連したライブラリ。

In [5]:
## まずはデコレーターの復習
def tag(f):
    def _wrapper(content):
        print('<h2>')
        r = f(content)
        print('</h2>')
        return r
    return _wrapper

@tag
def f(content):
    print(content)

f('test')

<h2>
test
</h2>


In [9]:
# ここからデコレーションを施す関数tagに引数を渡したい場合には、以下のように記述する。
def tag(name):
    def _tag(f):
        def _wrapper(content):
            print('<{}>'.format(name))
            r = f(content)
            print('</{}>'.format(name))
            return r
        return _wrapper
    return _tag

@tag('h3')
def f(content):
    print(content)

f('test')

<h3>
test
</h3>


In [2]:
# これをわかりやすくするために、contextlib.contextmanager
import contextlib

@contextlib.contextmanager
def tag(name):
    print('<{}>'.format(name))
    # yieldを使ってデコレーション対象の関数fを呼び出す。
    yield
    print('</{}>'.format(name))

@tag('h4')
def f(content):
    print(content)

f('test')

<h4>
test
</h4>


In [13]:
# 関数fを使わずに、withステートメントを使うことができる。
@contextlib.contextmanager
def tag(name):
    print('<{}>'.format(name))
    # yieldを使ってデコレーション対象の関数fを呼び出す。
    yield
    print('</{}>'.format(name))

with tag('h4'):
    print('test')


<h4>
test
</h4>


In [15]:
# withステートメントでネストのような形も作れる。
@contextlib.contextmanager
def tag(name):
    print('<{}>'.format(name))
    # yieldを使ってデコレーション対象の関数fを呼び出す。
    yield
    print('</{}>'.format(name))

def f():
    print('test0')
    with tag('h4'):
        print('test')
        with tag('h5'):
            print('test2')

f()

test0
<h4>
test
<h5>
test2
</h5>
</h4>


## contextlib.ContextDecorator
### 用途はcontextlib.contextmanagerとが、実装方法が異なる。ContextDecoratorを継承したクラスを定義して利用する。

In [8]:
# contextlib.ContextDecoratorを継承したクラスを定義
class tag(contextlib.ContextDecorator):
    def __init__(self, name):
        self.name = name
        self.start_tag = '<{}>'.format(name)
        self.end_tag = '</{}>'.format(name)

    # デコレーターに入った時の動作を定義
    def __enter__(self):
        print(self.start_tag)

    # デコレーターを抜けたときの動作を定義
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(self.end_tag)

with tag('h4'):
    print('test')

<h4>
test
</h4>


In [10]:
class tag(contextlib.ContextDecorator):
    def __init__(self, name):
        self.name = name
        self.start_tag = '<{}>'.format(name)
        self.end_tag = '</{}>'.format(name)

    def __enter__(self):
        print(self.start_tag)

    # Exceptionなどの情報が引数に入っている
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f'exc_type : {exc_type}')
        print(f'exc_val  : {exc_val}')
        print(f'exc_tb   : {exc_tb}')
        print(self.end_tag)

with tag('h4'):
    raise Exception('error')
    print('test')

<h4>
exc_type : <class 'Exception'>
exc_val  : error
exc_tb   : <traceback object at 0x0000020498C19988>
</h4>


Exception: error

## contextlib.supress

In [11]:
import os

try:
    # 存在しないファイル
    os.remove('somefile.tmp')
except FileNotFoundError:
    pass

In [12]:
# contextlib.suppressを使ってwithステートメントで、try-exceptと同じコードが書ける。
with contextlib.suppress(FileNotFoundError):
    os.remove('somefile.tmp')

## contextlib.exitstack

In [19]:
def is_ok_job():
    try:
        print('do something')
        raise Exception('error')
        return True
    except:
        return False

def cleanup():
    print('clean up')

try:
    is_ok = is_ok_job()
    print('more task')
finally:
    # is_okがTrueでなかった場合にcleanupの処理を実行する
    if not is_ok:
        cleanup()

do something
more task
clean up


このtry~finallyを置き換えるのが、contextlib.ExitStack  
以下のように使う。

In [1]:
import contextlib

def is_ok_job():
    try:
        print('do something')
        raise Exception('Error!')
        return True
    except Exception:
        return False


def cleanup():
    print('clean up')


with contextlib.ExitStack() as stack:
    # cleanupをスタックしておく。
    stack.callback(cleanup)

    is_ok = is_ok_job()
    print('more more more')

    # is_okがTrueの場合にpop_allでスタックを空にする。
    # もしFalseの場合はstackされたcleanupがコールバックされる。
    if is_ok:
        stack.pop_all()

do something
more more more
clean up


cleanup関数が引数を持っていて引数を渡す場合は、以下のように書ける。

In [2]:
import contextlib

def is_ok_job():
    try:
        print('do something')
        raise Exception('Error!')
        return True
    except Exception:
        return False


def cleanup(msg, *args, **kwargs):
    print(msg)
    print(args)
    print(kwargs)


with contextlib.ExitStack() as stack:
    # 引数を渡す場合は、以下のように渡す。
    stack.callback(cleanup, 'hello', 'good', 'ng', mysql='3306',
                   postgres='5432')

    is_ok = is_ok_job()
    print('more more more')

    if is_ok:
        stack.pop_all()

do something
more more more
hello
('good', 'ng')
{'mysql': '3306', 'postgres': '5432'}


また、withステートメントの中でインナー関数としてcleanup関数を記述できる。  
@stack.callbackをインナー関数に記述する

In [3]:
with contextlib.ExitStack() as stack:
    @stack.callback
    def inner_cleanup():
        print('inner cleanup')


    is_ok = is_ok_job()
    print('more more more')

    if is_ok:
        stack.pop_all()

do something
more more more
inner cleanup
